First install the repo and requirements.

In [ ]:
%pip --quiet install git+https://github.com/wilson-labs/cola.git

# Spectral Clustering

TODO: Add half moons examples

TODO: Improve text description below.

In this application, we'll show how `CoLA` can be used to perform Spectral Clustering. This application allows us to showcase how to use our `Sparse` operator.

As data, we will use the arXiv paper citation network of High Energy Physics. This is a directed graph, but we'll make it undirected by noting that if a paper cites another one, then they are related. To pre-process the data we will use the following code:

In [1]:
import pandas as pd
from scipy.sparse import csr_matrix
import numpy as np


def load_graph_data(filepath, dtype, xnp, num_edges=-1):
    df = pd.read_csv(filepath, skiprows=4, delimiter="\t", header=None, names=["to", "from"])
    df = df[:num_edges]
    df2 = pd.read_csv(filepath, skiprows=4, delimiter="\t", header=None, names=["from", "to"])
    df2 = df2[:num_edges]
    df_undir = pd.concat((df, df2), axis=0)
    df_undir = df_undir.drop_duplicates()
    id_map = map_nodes_to_id(df_undir["from"].unique())
    N = len(id_map)
    print(f"Found {N:,d} nodes")
    for col in ["from", "to"]:
        df_undir[col] = df_undir[col].map(id_map)
    data = np.ones(shape=len(df_undir))
    row, col = np.array(df_undir["to"]), np.array(df_undir["from"])
    sparse_matrix = csr_matrix((data, (row, col)), shape=(N, N))
    out = transform_to_csr(sparse_matrix, xnp, dtype)
    data, col_ind, rowptr, shape = out
    return data, col_ind, rowptr, shape

def transform_to_csr(sparse_matrix, xnp, dtype):
    data = xnp.array(sparse_matrix.data, dtype=dtype)
    indices = xnp.array(sparse_matrix.indices, dtype=xnp.int64)
    indptr = xnp.array(sparse_matrix.indptr, dtype=xnp.int64)
    return data, indices, indptr, sparse_matrix.shape


def map_nodes_to_id(nodes):
    out = {}
    for idx in range(len(nodes)):
        out[int(nodes[idx])] = idx
    return out

The function `load_graph_data` creates the column indices and row pointers needed for the sparse [CSR format](https://en.wikipedia.org/wiki/Sparse_matrix). Now, we can load the data and create our `Sparse` adjacency matrix as follows

In [2]:
import torch
import cola
from cola import torch_fns as xnp

filepath = "/home/pop/Downloads/cit-HepPh.txt"
num_edges = 100
dtype = torch.float64
data, col_ind, rowptr, shape = load_graph_data(filepath, dtype, xnp, num_edges)
Ad = cola.ops.Sparse(data, col_ind, rowptr, shape)

Found 84 nodes


/home/pop/cola/cola/ops/operators.py:56: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  self.A = self.ops.sparse_csr(indptr, indices, data)


Given the adjacency matrix, we can now create the normalized Laplacian defined as $L=I - D^{-1/2} A D^{-1/2}$, where $D$ is the diagonal matrix that contains the degree of each node, $A$ is the adjecency matrix and $I$ is the identity. We can create the Laplacian operator really easily in `CoLA` as

In [3]:
Deg = cola.ops.Diagonal(Ad @ xnp.ones(shape=(Ad.shape[0], ), dtype=dtype))
Id = cola.ops.I_like(Deg)
Lap = Id - cola.inverse(cola.sqrt(Deg)) @ Ad @ cola.inverse(cola.sqrt(Deg))
Lap = cola.SelfAdjoint(Lap)

Where I added the `SelfAdjoint` annotation at the end to ensure that `CoLA` dispatches algorithms for this type of symmetric operator. Spectral Clustering requires that we compute the eigenvectors of the smallest eigenvalues and use those eigenvectors as an embedding of our data. Once we do this, we can then use k-means to cluster points nearby as those points are related to cliques in the original graph.

In [4]:
from sklearn.cluster import KMeans

embedding_size, n_clusters = 8, 8
eigvals, eigvecs = cola.eig(Lap, method="lanczos")
x_emb = eigvecs[:, :embedding_size].to_dense()
kmeans = KMeans(n_clusters=n_clusters).fit(x_emb)
print(kmeans.labels_)

[2 4 4 4 4 4 1 0 4 2 2 4 4 4 4 4 4 4 4 0 0 0 2 0 0 0 2 1 1 1 7 1 1 1 1 1 1
 2 2 2 2 2 6 5 5 3 5 5 7 5 5 6 5 5 6 6 6 6 7 7 7 7 7 7 7 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 4 5]


/home/pop/venv/cola/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
